<h1>Location Extractor</h1>
<p>This nb was used to extract the location data using the name of each GN division.<br>
Google map API was used with geopy to extract the location data of each location.<br>
Google map API was removed from the file. So, there wan't be any issue with shareing the file.</p>

In [54]:
# import packages
import pandas as pd
from geopy import GoogleV3

In [109]:
# read the csv file to df
colomboCensus = pd.read_csv('colomboCensus.csv')
colomboCensus.head()

,DS,GN,GN Number,Both sexes,Male,Female
0,Colombo,Sammanthranapura,NaN,"7,829","4,017","3,812"
1,Colombo,Mattakkuliya,NaN,"28,003","14,029","13,974"
2,Colombo,Modara,NaN,"17,757","8,794","8,963"
3,Colombo,Madampitiya,NaN,"12,970","6,505","6,465"
4,Colombo,Mahawatta,NaN,"8,809","4,367","4,442"


In [116]:
# google maps API key
APIKEY = 'Input the API key here'

In [14]:
# define a list to store the outputs
colomboLocations = []
# define a Nominatum object to get location data
googleMap = GoogleV3(APIKEY)
# get the location data of each location add to df
for brough, neighborhood in zip(colomboCensus['DS'], colomboCensus['GN']):
    address = '{}, {}'.format(neighborhood, brough)
    row = {}
    # get location data using geopy
    location = googleMap.geocode(address, region='LK')
    # populate the row dict with extracted data
    row['DS'] = brough
    row['GN'] = neighborhood
    if location is not None:
        row['Latitude'] = location.latitude
        row['Longitude'] = location.longitude
    else:
        row['Latitude'] = 0
        row['Longitude'] = 0
    # add the dict to list
    colomboLocations.append(row)

In [110]:
# turn the location data into a df
tempDF = pd.DataFrame(colomboLocations)
# drop the DS column from df
tempDF.drop('DS', axis=1, inplace=True)

In [111]:
# change the dtypes of df columns befor join
# check the dtypes
colomboCensus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 557 entries, 0 to 556
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   DS          557 non-null    object
 1   GN          557 non-null    object
 2   GN Number   502 non-null    object
 3   Both sexes  557 non-null    object
 4   Male        557 non-null    object
 5   Female      557 non-null    object
dtypes: object(6)
memory usage: 26.2+ KB


In [112]:
# remove comas from the numeric values
colomboCensus.replace(',', '', regex=True, inplace=True)
# change the dtypes
colomboCensus = colomboCensus.astype({'Both sexes':'int32', 'Male':'int32', 'Female':'int32'})
# check the dtype after modification
colomboCensus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 557 entries, 0 to 556
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   DS          557 non-null    object
 1   GN          557 non-null    object
 2   GN Number   502 non-null    object
 3   Both sexes  557 non-null    int32 
 4   Male        557 non-null    int32 
 5   Female      557 non-null    int32 
dtypes: int32(3), object(3)
memory usage: 19.7+ KB


In [113]:
# check the df dtypes
tempDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 557 entries, 0 to 556
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   GN         557 non-null    object 
 1   Latitude   557 non-null    float64
 2   Longitude  557 non-null    float64
dtypes: float64(2), object(1)
memory usage: 13.2+ KB


In [114]:
# join two df
colomboCensus = colomboCensus.merge(tempDF, on='GN')
colomboCensus.head()

,DS,GN,GN Number,Both sexes,Male,Female,Latitude,Longitude
0,Colombo,Sammanthranapura,NaN,7829,4017,3812,6.978943,79.877983
1,Colombo,Mattakkuliya,NaN,28003,14029,13974,6.971672,79.878683
2,Colombo,Modara,NaN,17757,8794,8963,6.966925,79.871153
3,Colombo,Madampitiya,NaN,12970,6505,6465,6.961663,79.875184
4,Colombo,Mahawatta,NaN,8809,4367,4442,6.958307,79.873084


In [115]:
# store the results as a csv file
colomboCensus.to_csv('colomboCensusProcessed.csv', index=False)